In [19]:
import json
import requests
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def get_uniswap_graphql(payload) -> pd.DataFrame:
    try:
        r = requests.post(
            'https://api.thegraph.com/subgraphs/name/1hive/uniswap-v2',
            data=json.dumps(payload),
            timeout=30.0
        )
        print(f"thegraph status_code: {r}")
        r1 = r.json()
        if r1.get('data'):
            return pd.DataFrame(r1['data']['pairs'])
        else:
            raise ValueError('no data in response')
    except Exception as e:
        print(f"doh: {e}")
 
graphql_payload_1st1000 = {
    "operationName": "pairs",
    "variables": {},
    "query":"query pairs {\n pairs(first: 1000) {\n id token0 {\n symbol\n name\n id\n decimals\n \n}\n token1 {\n symbol\n name\n id\n decimals\n \n}\n reserve0 reserve1 \n} \n}\n"
}

df = get_uniswap_graphql(graphql_payload_1st1000)
print(len(df))
# pair_addr: str combining both addresses for the purpose of detecting duplicates pairs.
df['pair_addr'] = [f"{x['id']}/{y['id']}" for x, y in zip(df['token0'], df['token1'])]
# pair_symbol: str combining both symbols for the purpose of detecting duplicates pairs.
df['pair_symbol'] = [f"{x['symbol']}/{y['symbol']}" for x, y in zip(df['token0'], df['token1'])]
df.head()

thegraph status_code: <Response [200]>
882


,id,reserve0,reserve1,token0,token1,pair_addr,pair_symbol
0,0x002b85a23023536395d98e6730f5a5fe8115f08b,0.000127493510283084,0.016367798175231776,"{'decimals': '18', 'id': '0x71850b7e9ee3f13ab46d67167341e4bdc905eef9', 'name': 'Honey', 'symbol': 'HNY'}","{'decimals': '18', 'id': '0x9c374cc5da1d7ee668d4080287b104b865f3089e', 'name': 'AMISDEXP', 'symbol': 'DEXP'}",0x71850b7e9ee3f13ab46d67167341e4bdc905eef9/0x9c374cc5da1d7ee668d4080287b104b865f3089e,HNY/DEXP
1,0x00892cf46a56d78328f03a9cb9361a02abc47511,85488.988743912020283322,1.772824058241431257,"{'decimals': '18', 'id': '0x2f9cebf5de3bc25e0643d0e66134e5bf5c48e191', 'name': 'Bricks on xDai', 'symbol': 'xBRICK'}","{'decimals': '18', 'id': '0x71850b7e9ee3f13ab46d67167341e4bdc905eef9', 'name': 'Honey', 'symbol': 'HNY'}",0x2f9cebf5de3bc25e0643d0e66134e5bf5c48e191/0x71850b7e9ee3f13ab46d67167341e4bdc905eef9,xBRICK/HNY
2,0x009d7fb78f2fb1fe6eff81be3136d02faf46db4d,0.0183615627848776,460.18820079,"{'decimals': '18', 'id': '0x71850b7e9ee3f13ab46d67167341e4bdc905eef9', 'name': 'Honey', 'symbol': 'HNY'}","{'decimals': '8', 'id': '0xd7738ec74f08f3a14c478bbecce9c3f5603ba458', 'name': 'Dogecoin on xDai', 'symbol': 'DOGE'}",0x71850b7e9ee3f13ab46d67167341e4bdc905eef9/0xd7738ec74f08f3a14c478bbecce9c3f5603ba458,HNY/DOGE
3,0x0172155c565e783c51b96b608316d489b9bd7463,116.566588612344918231,3.075280774755157096,"{'decimals': '18', 'id': '0x759a891d86436737fca1fad68e6d77b1198bb40d', 'name': 'TEST', 'symbol': 'TEST'}","{'decimals': '18', 'id': '0xe91d153e0b41518a2ce8dd3d7944fa863463a97d', 'name': 'Wrapped XDAI', 'symbol': 'WXDAI'}",0x759a891d86436737fca1fad68e6d77b1198bb40d/0xe91d153e0b41518a2ce8dd3d7944fa863463a97d,TEST/WXDAI
4,0x01f4a4d82a4c1cf12eb2dadc35fd87a14526cc79,47875.192845,48225.689212763644244176,"{'decimals': '6', 'id': '0xddafbb505ad214d7b80b1f830fccc89b60fb7a83', 'name': 'USD//C on xDai', 'symbol': 'USDC'}","{'decimals': '18', 'id': '0xe91d153e0b41518a2ce8dd3d7944fa863463a97d', 'name': 'Wrapped XDAI', 'symbol': 'WXDAI'}",0xddafbb505ad214d7b80b1f830fccc89b60fb7a83/0xe91d153e0b41518a2ce8dd3d7944fa863463a97d,USDC/WXDAI


In [20]:
df[df.duplicated(['pair_addr'], keep=False)]

,id,reserve0,reserve1,token0,token1,pair_addr,pair_symbol


In [22]:
df_dup = df[df.duplicated(['pair_symbol'], keep=False)]
print(len(df_dup))
df_dup[['pair_symbol', 'id', 'token0', 'token1']].sort_values('pair_symbol',  ascending=False)

96


,pair_symbol,id,token0,token1
370,zzz/HNY,0x682302b7dff726819b189d5c161aea7ca331b2d4,"{'decimals': '18', 'id': '0x32bc30165b6059cadb6ae14423f63817e2db12ae', 'name': 'zzz', 'symbol': 'zzz'}","{'decimals': '18', 'id': '0x71850b7e9ee3f13ab46d67167341e4bdc905eef9', 'name': 'Honey', 'symbol': 'HNY'}"
596,zzz/HNY,0xa7967209efb10e42a9d0c6c5b4a585cc44769264,"{'decimals': '18', 'id': '0x15461ca3cfa87b69b644f9504e4b64cde7331918', 'name': 'zzz', 'symbol': 'zzz'}","{'decimals': '18', 'id': '0x71850b7e9ee3f13ab46d67167341e4bdc905eef9', 'name': 'Honey', 'symbol': 'HNY'}"
505,xdbx/WXDAI,0x8f6b98cc0ab15486b344393333c20d988e0e2071,"{'decimals': '18', 'id': '0x10d7116422d3ad15cff6e2ddd0ad5b640b14d426', 'name': 'xDankBillz', 'symbol': 'xdbx'}","{'decimals': '18', 'id': '0xe91d153e0b41518a2ce8dd3d7944fa863463a97d', 'name': 'Wrapped XDAI', 'symbol': 'WXDAI'}"
500,xdbx/WXDAI,0x8e2a73b43a1bc3093f8c96a41d43c095de79a8a1,"{'decimals': '18', 'id': '0xdaadd8d96d01e47ee5e4eafecf14cbe46909f335', 'name': 'xDankBillz', 'symbol': 'xdbx'}","{'decimals': '18', 'id': '0xe91d153e0b41518a2ce8dd3d7944fa863463a97d', 'name': 'Wrapped XDAI', 'symbol': 'WXDAI'}"
407,wTDAO/WXDAI,0x71a5d0a0801544894d94a6056d459c4854ab68de,"{'decimals': '18', 'id': '0xd1a1afd803339e4eaa74b7b42448bebb137f1bde', 'name': 'Wrapped Contribute DAO', 'symbol': 'wTDAO'}","{'decimals': '18', 'id': '0xe91d153e0b41518a2ce8dd3d7944fa863463a97d', 'name': 'Wrapped XDAI', 'symbol': 'WXDAI'}"
682,wTDAO/WXDAI,0xc122159f470ec9cac83c92894bcb783680d78e40,"{'decimals': '18', 'id': '0x4609e9b9c2912dd5b954cbf3a5d7d89ab6c8979d', 'name': 'Wrapped Contribute DAO', 'symbol': 'wTDAO'}","{'decimals': '18', 'id': '0xe91d153e0b41518a2ce8dd3d7944fa863463a97d', 'name': 'Wrapped XDAI', 'symbol': 'WXDAI'}"
386,wTDAO/WXDAI,0x6be70a54656e388e69b615b30d74b4df548f361b,"{'decimals': '18', 'id': '0x9efae005fb3736e48e581ed1dd0d669c64ff9c6c', 'name': 'Wrapped Contribute DAO', 'symbol': 'wTDAO'}","{'decimals': '18', 'id': '0xe91d153e0b41518a2ce8dd3d7944fa863463a97d', 'name': 'Wrapped XDAI', 'symbol': 'WXDAI'}"
744,testCODE/WXDAI,0xd3e3eda4c28260ceea029805d780fbc7709cc58a,"{'decimals': '18', 'id': '0x50e1b967b1940464dfe1eb08241e8b01041a1e9f', 'name': 'testCODE', 'symbol': 'testCODE'}","{'decimals': '18', 'id': '0xe91d153e0b41518a2ce8dd3d7944fa863463a97d', 'name': 'Wrapped XDAI', 'symbol': 'WXDAI'}"
830,testCODE/WXDAI,0xf0f234520365ac3797ee8aa63f5e6bbe0e2e10a1,"{'decimals': '18', 'id': '0xae9bc90a5309139d3c159436ed3b61b5ff99e36f', 'name': 'testCODE', 'symbol': 'testCODE'}","{'decimals': '18', 'id': '0xe91d153e0b41518a2ce8dd3d7944fa863463a97d', 'name': 'Wrapped XDAI', 'symbol': 'WXDAI'}"
585,symbol/HNY,0xa41947e487b9e35a515be82c8148d7183cbebfcb,"{'decimals': '18', 'id': '0x67f1fbe173c2a2e9a3e21fa0d613f866fe13bf46', 'name': 'name', 'symbol': 'symbol'}","{'decimals': '18', 'id': '0x71850b7e9ee3f13ab46d67167341e4bdc905eef9', 'name': 'Honey', 'symbol': 'HNY'}"
